# DOCUMENTATION AND NOTES:

*** Everything used: ***

Streamlit will be used to build the app: https://streamlit.io

Diffusion model pretrained pipelines:
* https://huggingface.co/CompVis/stable-diffusion-v1-4
* https://huggingface.co/stabilityai/stable-diffusion-2-1

StudioGAN will be used for pretrained GAN models: https://github.com/POSTECH-CVLab/PyTorch-StudioGAN (Not used)

One of the GAN models is FuseDream (CLIP + BigGAN):
 * article: https://analyticsindiamag.com/fusedream-a-hands-on-tutorial-on-this-text-to-image-generation-tool/
 * author repo: https://github.com/gnobitab/FuseDream
 * BigGAN repo: https://github.com/huggingface/pytorch-pretrained-BigGAN

Dreambooth:
* Used BIRME for resizing images to 512x512: https://www.birme.net/?target_width=512&target_height=512

* Used Dreambooth by Shivam Shrirao diffuser based repo: https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth (examples/dreambooth/DreamBooth_Stable_Diffusion.ipynb)






-------------------------

Add features and qualities needed:


8) Secondary page for showing the different model outputs. (REMOVED)

   * (The outputs can be displayed in a carousel manner that shows which model is used on top left.) (use st.tabs while doing this)

9) Red background with black and white text for general Akbank asthetic. (Sidebar title and headers and the general page title should be turned to white.)

10) Area for viewing general model information and if desired, tweaking the parameters.)
  
   * (The areas are done however, general information regarding the models should be filled for all models.)
   
-------------------------

* On the computation side, look up: 384x384 generation to superresolution (DLSS) and neural style transfer.



Taskbar

-------------------------
POSSIBLE ADDITIONS FOR HIGHER QUALITY:

1) Add scheduler selector:

* Add K-LMS scheduler for SD1-4.

* Add DPMSolverMultistep scheduler for SD2-1.

2) Look up more parameters to further optimize stable diffusion.

3) Add VQGAN + Clıp for variation:

* Article: https://learn.adafruit.com/generating-ai-art-with-vqgan-clip/basic-use

* Google Colab: https://colab.research.google.com/drive/1go6YwMFe5MX6XM9tv-cnQiSTU50N9EeT#scrollTo=VA1PHoJrRiK9

4) Output three generated images (?)

5) Instead of dreambooth try out LoRA for the fine-tuning for faster results and easier sharing, storage, and re-use. (LoRA is an improved finetuning method where instead of finetuning all the weights that constitute the weight matrix of the pre-trained large language model, two smaller matrices that approximate this larger matrix are fine-tuned.)

-------------------------

REMAINING:

FlowBMs, implement their models, create and connect their parameters.


IMPORTANT:

There is a CUDA memory problem because we are trying to import too many things.

Placeholder solution: Import modules when you need, within the functions that are called based on model. (Implemented)

Better solution: Import modules like above but unless the next model is the same type that requires the same modules delete the modules within the functions. (Need to keep track of the previous model type)  

! Could not implement the above, deletion can only occur within functions when the function is called.

GANS //


Stuff to try out:

* BigGAN: Is a large-scale GAN model developed by Google. It's designed for generating high-resolution images conditioned on text descriptions or class labels.

* StyleGAN:  (Style Generative Adversarial Network) is a generative model architecture designed for generating high-resolution and highly realistic images, particularly faces and artworks.

 * StyleGAN2, which refines the architecture and training methods, and StyleGAN2-ADA (Adaptive Discriminator Augmentation), which further enhances training stability and image quality.

* CLIP+GAN: Combining CLIP with GANs, you can condition the generation process on textual prompts and achieve text-to-image synthesis with improved context awareness.

* AttnGAN: Attention Generative Adversarial Network (AttnGAN) is designed explicitly for fine-grained text-to-image generation. It uses attention mechanisms to focus on different parts of the text description while generating corresponding image details.

* TAC-GAN: Text Conditioned Auxiliary Classifier Generative Adversarial Network, presents the study of generating realistic images through text-to-image synthesis with the help of GANs.

-------------------------

Possible GANs to implement: (Especially look at Stacked/Stack GANs if you have time.)

* Stacked GAN: Stacked GAN refers to a general concept of stacking multiple GANs or GAN-like models on top of each other to improve the quality of generated images.

  * StackGAN and StackGAN++: StackGAN is a particular implementation of stacked GANs designed for the task of text-to-image synthesis. It was developed to generate photo-realistic images from textual descriptions.

  * Can use a StackGAN when you need to generate images from a completely different representation (e.g., from text-based descriptions).
  *  authors repo: https://github.com/hanzhanggit/StackGAN

* Information Maximizing GAN (Not verry useful for this specific task but can be fitted if needed.)

* Super Resolution GAN:

  *  Can use an SRGAN when you need to upscale images while recovering or preserving fine-grain, high-fidelity details. (Can be used to initially generate low dimensional images with other generational models, then be used to upscale them (?))

-------------------------

If everything else fails: pixelRNN, text-2-image, DiscoGAN, and IsGAN.


FLOW BASED GENERATION MODELS //

* Glow
* RealNVP

*** Dreambooth: Ideal parameters for cartoonish avatar generation: ***

* for 8 training images

  with_prior_preservation

  train_batch_size=2 (effect not tested)

  train_text_encoder

  mixed_precision="fp16"

  use_8bit_adam

  gradient_accumulation_steps=1

  learning_rate=1e-6

  num_class_images=100 (effect not tested)

  sample_batch_size=4 (effect not tested)

  max_train_steps=640 and 664
  
   * Previously tried 700 (and higher, untill 1200), presents a high risk of overfitting and every 1 image out of 5 is unusable. As of now the ideal number of training steps seems to be *** 80-83 epochs per image ***.

  save_interval=10000

  save_sample_prompt="yigitya person"

  ------------------------------------------------

  prompt = "A cartoonish avatar version of yigitya person."

  negative_prompt = "weird eyes, blurry eyes"

  num_samples = 1

  guidance_scale = 9

  num_inference_steps = 100

 ------------------------------------------------

 Accurate filenames were used, file names identical to instance prompt yigitya person. Also tried unrelated file names, ***does not seem to make a difference.***
    

# IMPLEMENTATION:

In [1]:
# For application and stable diffusion models:
!pip install streamlit -q
!pip install --upgrade diffusers transformers scipy
!pip install diffusers transformers accelerate scipy safetensors

  Obtaining dependency information for diffusers from https://files.pythonhosted.org/packages/2e/ed/58a13f88dfcdd1bcfeabf44d4c9861979551339348e579fd6559c64b12e0/diffusers-0.21.4-py3-none-any.whl.metadata
  Using cached diffusers-0.21.4-py3-none-any.whl.metadata (18 kB)
Using cached diffusers-0.21.4-py3-none-any.whl (1.5 MB)
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.15.0.dev0
    Uninstalling diffusers-0.15.0.dev0:
      Successfully uninstalled diffusers-0.15.0.dev0


In [2]:
# For GAN models:
!git clone https://github.com/gnobitab/FuseDream.git
!pip install ftfy regex tqdm numpy scipy h5py lpips==0.1.4
!pip install git+https://github.com/openai/CLIP.git
!pip install gdown
!gdown 'https://drive.google.com/uc?id=1sOZ9og9kJLsqMNhaDnPJgzVsBZQ1sjZ5'

fatal: destination path 'FuseDream' already exists and is not an empty directory.
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/1r/dx6fjhj10k37w5dnl4mrmvhc0000gn/T/pip-req-build-xxeud83_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/1r/dx6fjhj10k37w5dnl4mrmvhc0000gn/T/pip-req-build-xxeud83_
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


Downloading...
From (uriginal): https://drive.google.com/uc?id=1sOZ9og9kJLsqMNhaDnPJgzVsBZQ1sjZ5
From (redirected): https://drive.google.com/uc?id=1sOZ9og9kJLsqMNhaDnPJgzVsBZQ1sjZ5&confirm=t&uuid=8b553a01-2e17-4f37-8339-98d5a9a0201e
To: /Users/berkedenizbozyigit/Downloads/biggan-512.pth
100%|████████████████████████████████████████| 330M/330M [00:32<00:00, 10.0MB/s]


In [3]:
!ls
!cp biggan-512.pth FuseDream/BigGAN_utils/weights/
%cd FuseDream

5x555555.JPG
Antrag auf Immatrikulation.pdf
CATS_DOGS.zip.download
Doktar_Case_Study
Doktar_Case_Study_Report-2.pdf
Doktar_Case_Study_Report-3.pdf
Doktar_Case_Study_Report.pdf
Doktar_Machine_Learning_Engineer_Case_Study
Eczacibasi_Wed Aug 09 2023 10_32_04 GMT+0300 (GMT+03_00).pdf
FuseDream
Image_Generation_UI_Project.ipynb
Install Spotify.app
Machine_Learning_Engineer_Case_Study.pdf
Recomendation letter 2.pdf
Recommendation letter.pdf
Sublime Text.app
TF_2_Notebooks_and_Data
TF_2_Notebooks_and_Data-2
Transcript.pdf
UK (1).docx
UK.docx
UNZIP_FOR_NOTEBOOKS_FINAL
UNZIP_FOR_NOTEBOOKS_FINAL-2
University courses.pdf
Unknown
Unknown-2
Unknown-3
Visual Studio Code.app
WhatsApp Unknown 2023-09-17 at 20.03.23.zip
WhatsApp Unknown 2023-09-17 at 20.03.52
WhatsApp Unknown 2023-09-17 at 20.03.52.zip
YAPILMASI GEREKENLER.xlsx
YORK OFFER DOCS
akbank.jpeg
akbank.jpg
biggan-512.pth
doktor_case_study_new.py
dvdrental.tar
exercises.tar
googlechrome.dmg
kc_house_data.csv
myspark.pem
pgadmin4-7.5-x86_64.dmg

In [4]:
# Dreambooth requirements:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton                                                 # use either this or the one at the bottom
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers

zsh:1: command not found: wget
zsh:1: command not found: wget
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement triton (from versions: none)
ERROR: No matching distribution found for triton
Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [229 lines of output]
      running bdist_wheel
      /Applications/anaconda3/envs/Avatar/lib/python3.11/site-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
        warnings.warn(msg.format('we could not find ninja.'))
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-311
      creating build/lib.macosx-11.1-arm64-cpython-311/xformers

ERROR: Could not build wheels for xformers, which is required to install pyproject.toml-based projects
Note: you may need to restart the kernel to use updated packages.


In [5]:
!accelerate config default

accelerate configuration saved at /Users/berkedenizbozyigit/.cache/huggingface/accelerate/default_config.yaml


In [7]:
# V3: (Final Version)  -->  (Fix issue: Additional fine-tuning information within the app has red underline when deployed.)

writefile akbankavatargenerator.py
#####################################
import streamlit as st
import base64
import torch
from PIL import Image
#####################################
# Functions:

# Function for calling any of the two stable diffusion models, imports and deletes required modules, parameters are self explanatory, returns the image:
def stablediffusioncall(option, user_prompt, user_n_prompt, i_inference_steps, i_initial_seed, i_guidance_scale, situation):
  from diffusers import StableDiffusionPipeline

  if option == "Stable Diffusion v1-4":        # Uses a PNDM scheduler
    model_id = "CompVis/stable-diffusion-v1-4"
  elif option == "Stable Diffusion 2-1":
    model_id = "stabilityai/stable-diffusion-2-1" # Uses a DDIM scheduler

  device = "cuda"
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
  pipe = pipe.to(device)

  prompt = user_prompt
  negative_prompt = user_n_prompt

  if situation:     # If custom parameters are chosen, parameters are given the user input values. If not, the pipeline is called without change.
    num_inference_steps = i_inference_steps
    guidance_scale = i_guidance_scale

    if i_initial_seed is not None:        # In case of wanting to use custom seed, use the user input value. If not the pipeline is called without seed input.
      generator = torch.Generator("cuda").manual_seed(i_initial_seed)
      image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, negative_prompt=negative_prompt, generator=generator ).images[0]
    else:
      image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, negative_prompt=negative_prompt).images[0]
  else:
    image = pipe(prompt=prompt, negative_prompt=negative_prompt).images[0]

  del StableDiffusionPipeline
  return image

# Function for calling FuseDream model, imports and deletes required modules, parameters are self explanatory, returns the image:
def fusedreamcall(user_prompt, INIT_ITERS, OPT_ITERS, NUM_BASIS, SEED, use_seed):
  from tqdm import tqdm
  from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
  import torchvision
  import BigGAN_utils.utils as utils
  import clip
  import torch.nn.functional as F
  from DiffAugment_pytorch import DiffAugment
  import numpy as np
  from fusedream_utils import FuseDreamBaseGenerator, get_G
  import sys

  sentence = user_prompt

  if use_seed:           # If the function is called using default parameter values SEED and use_seed are given the value None. If the utils.seed_rng(SEED) is called with None it gives an error, therefore the if statment is used.
    utils.seed_rng(SEED)

  sys.argv = ['']

  G, config = get_G(512)
  generator = FuseDreamBaseGenerator(G, config, 10)
  z_cllt, y_cllt = generator.generate_basis(sentence, init_iters=INIT_ITERS, num_basis=NUM_BASIS)

  z_cllt_save = torch.cat(z_cllt).cpu().numpy()
  y_cllt_save = torch.cat(y_cllt).cpu().numpy()
  img, z, y = generator.optimize_clip_score(z_cllt, y_cllt, sentence, latent_noise=False, augment=True, opt_iters=OPT_ITERS, optimize_y=True)

  image = img
  image = (image / 2 + 0.5).clamp(0, 1)
  image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
  images = (image * 255).round().astype("uint8")
  pil_images = [Image.fromarray(image) for image in images]
  image = pil_images[0]

  del tqdm
  del torchvision
  del utils
  del clip
  del F
  del DiffAugment
  del FuseDreamBaseGenerator
  del get_G
  return image

# Function for fine-tuning stable diffusion v1-5 model, imports and deletes required modules, parameters are self explanatory, returns the fine-tuned model pipeline:
def finetunesdcall(u_token, u_class, plant, t_b_size, l_r, n_c_images, s_b_size, m_t_steps, uploaded_files):
  # Settings (including model selection):

  import subprocess

  s_prompt=u_token+" "+u_class

  MODEL_NAME = "runwayml/stable-diffusion-v1-5"

  OUTPUT_DIR = "/content/stable_diffusion_weights/" + u_token

  try:
      # Construct the shell command
      command = f"mkdir -p {OUTPUT_DIR}"
      # Execute the shell command
      subprocess.run(command, shell=True, check=True)
  except subprocess.CalledProcessError as e:
      # Handle any errors that occur during command execution
      print(f"Error: {e}")

  # Choosing the instance and class prompts:

  # You can also add multiple concepts here, try tweaking `--max_train_steps` accordingly.
  # `class_data_dir` contains the regularization images
  concepts_list = [
      {
          "instance_prompt":      f"photo of {u_token} man",
          "class_prompt":         "photo of a man",
          "instance_data_dir":    "/content/data/" + u_token,
          "class_data_dir":       "/content/data/man"
      },
      {
          "instance_prompt":      f"photo of {u_token} woman",
          "class_prompt":         "photo of a woman",
          "instance_data_dir":    "/content/data/" + u_token,
          "class_data_dir":       "/content/data/woman"
      },
      {
          "instance_prompt":      f"photo of {u_token} person",
          "class_prompt":         "photo of a person",
          "instance_data_dir":    "/content/data/" + u_token,
          "class_data_dir":       "/content/data/person"
      }
  ]

  import json
  import os
  for c in concepts_list:
      os.makedirs(c["instance_data_dir"], exist_ok=True)

  keep_cl = []
  for c in concepts_list:
    a_class = c['instance_prompt'].split()
    if a_class[-1] == u_class:
      keep_cl.append(c)

  with open("concepts_list.json", "w") as f:
      json.dump(keep_cl, f, indent=4)

  # Upload your images

  from google.colab import files
  import shutil

  for c in concepts_list:
    a_class = c['instance_prompt'].split()
    if a_class[-1] == u_class:
      uploaded = uploaded_files
      for element in uploaded:
        filename = element.name
        dst_path = os.path.join(c['instance_data_dir'], filename)
        with open(dst_path, "wb") as f:
            f.write(element.read())

  # Fine-tuning the SD model:

  # Tweak the parameters for desired image quality --> 1200 training steps = 40 newly introdued images for 30 epochs.
  command = f"""
          python3 train_dreambooth.py \
          --pretrained_model_name_or_path={MODEL_NAME} \
          --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
          --output_dir={OUTPUT_DIR} \
          --revision="fp16" \
          --with_prior_preservation --prior_loss_weight=1.0 \
          --seed={plant} \
          --resolution=512 \
          --train_batch_size={t_b_size} \
          --train_text_encoder \
          --mixed_precision="fp16" \
          --use_8bit_adam \
          --gradient_accumulation_steps=1 \
          --learning_rate={l_r} \
          --lr_scheduler="constant" \
          --lr_warmup_steps=0 \
          --num_class_images={n_c_images} \
          --sample_batch_size={s_b_size} \
          --max_train_steps={m_t_steps} \
          --save_interval=10000 \
          --save_sample_prompt="{s_prompt}" \
          --concepts_list="concepts_list.json"
      """
  try:
      # Execute the shell command
      subprocess.run(command, shell=True, check=True, executable="/bin/bash")
  except subprocess.CalledProcessError as e:
      # Handle any errors that occur during command execution
      print(f"Error: {e}")

  # Reduce the `--save_interval` to lower than `--max_train_steps` to save weights from intermediate steps.
  # `--save_sample_prompt` can be same as `--instance_prompt` to generate intermediate samples (saved along with weights in samples)

  # Specify the weights directory to use (leave blank for latest):

  from natsort import natsorted
  from glob import glob
  WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]

  # Inference:

  import torch
  from torch import autocast
  from diffusers import StableDiffusionPipeline, DDIMScheduler
  from IPython.display import display

  model_path = WEIGHTS_DIR

  pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
  pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
  pipe.enable_xformers_memory_efficient_attention()
  g_cuda = None

  del json
  del os
  del files
  del shutil
  del natsorted
  del glob
  del torch
  del autocast
  del display
  del subprocess
  return pipe

# Function for generating prompt conditioned images with fine-tuned stable diffusion v1-5 model, imports and deletes required modules, parameters are self explanatory, returns the image:
def dbgeneratecall(pipe, user_prompt, user_n_prompt, guidance_scale, num_inference_steps, use_seed, s33d):

  import torch
  from torch import autocast
  from diffusers import StableDiffusionPipeline, DDIMScheduler

  # Run for generating images.

  prompt = user_prompt
  negative_prompt = user_n_prompt
  num_samples = 1
  height = 512
  width = 512

  if use_seed:
    g_cuda = torch.Generator(device='cuda')
    seed = s33d
    g_cuda.manual_seed(seed)
    with autocast("cuda"), torch.inference_mode():
        images = pipe(
            prompt,
            height=height,
            width=width,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_samples,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=g_cuda
        ).images
  else:
    with autocast("cuda"), torch.inference_mode():
        images = pipe(
            prompt,
            height=height,
            width=width,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_samples,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
        ).images

  for img in images:
    generated = img

  del torch
  del autocast
  return generated

###############################################################################################################

def main():

  icon = Image.open('/content/icon.png')
  st.set_page_config(page_icon = icon, layout="wide")

  #background_image = Image.open('/content/background.jpg')

  col1, coli1, col2, coli2, col3, = st.columns((1,0.25,2,0.25,1))

  image = Image.open('/content/akbank.jpg')
  image2 = Image.open('/content/sabancı.png')

  if 'files_uploaded' not in st.session_state:  # Keeps track of if any image has been uploaded (keeps boolean value). Initializing it beforehand, prevents being called before being initialized error
    st.session_state.files_uploaded = False

  if 'pictures' not in st.session_state:  # Keeps track of if any image has been uploaded (keeps the pictures themselves). Initializing it beforehand, prevents being called before being initialized error
    st.session_state.pictures = False

  if 'pipe_s' not in st.session_state:  # Keeps track of if pipe_s has been returned by finetunesdcall (keeps the pipeline). Initializing it beforehand, prevents being called before being initialized error
    st.session_state.pipe_s = False

  if 'fine_tuned' not in st.session_state:  # Keeps track of if the model has been fine-tuned. Initializing it beforehand, prevents being called before being initialized error
    st.session_state.fine_tuned = False

  with col1:
    st.image(image, caption='', width = 500, use_column_width=True)
    with st.sidebar:
      st.title(':red[MODELS]')
      st.header('General model information', divider='red')

      with st.expander("Stable Diffusion v1-4"):
        st.write("PLaceholder.")

      with st.expander("Stable Diffusion 2-1"):
        st.write("PLaceholder.")

      with st.expander("FuseDream (CLIP+BigGAN)"):
        st.write("PLaceholder.")

      with st.expander("Non-specified Flow based model"):
        st.write("PLaceholder.")

      st.header('Settings to tweak the models', divider='red')
      # situation = True --> Want to use custom parameters, situation = False --> Want to use default parameters
      situation = st.toggle('Use custom parameters for models')
      if situation:
        with st.expander('Diffusion model parameters:'):  # Have 3 parameters

          i_inference_steps = st.number_input('Number of inference steps', step = 1)

          use_seed = st.toggle('Enter a seed value', help = 'If seed value not entered, each generated image will be given a random one.')
          if use_seed:
            i_initial_seed = st.number_input('Initial seed', value = 0, step = 1)
          else:
            i_initial_seed = None

          i_guidance_scale = st.slider('Guidance scale', 1.0, 25.0, 7.5, step = 0.5)

        with st.expander('FuseDream (CLIP+BigGAN) parameters:'): # Has 5 parameters (including the boolean use_seed)

          INIT_ITERS = st.number_input('Number of images used for initialization', step = 1)

          OPT_ITERS = st.number_input('Number of iterations', step = 1)

          NUM_BASIS = st.number_input('Number of basis images', step = 1)

          use_seed = st.toggle('Enter a seed value', key = 4, help = 'If seed value not entered, each generated image will be given a random one.')
          if use_seed:
            SEED = st.number_input('Initial seed', value = 0, step = 1, key = 5)
          else:
            SEED = None

        with st.expander('Non-specified Flow based model parameters:'): # Not yet implemented, has 0 parameters
          st.write("PLaceholder.")

      st.header(' ', divider='red')
      # Dreambooth is separated from the situation block since it should not be called with default parameters as the parameters need to be entered according to the external data and its use. (TLDR Attune parameters per each use for good results.)
      with st.expander('Dreambooth parameters:'): # Has 8 parameters for fine-tuning and 4 parameters for image generation (including use_seed boolean)

        u_token = st.text_input('Unique token', '', placeholder='Enter a unique token.', help = 'example: name_surname or zwx')

        u_class = st.text_input('Class of reg images', '', placeholder='Enter a regularization image set class.', help = 'Available class names: man, woman, person')

        plant = st.number_input('Initial seed', value = 0, step = 1, key = 3)

        t_b_size = st.number_input('Training bach size', step =1, key = 1)

        l_r = st.slider('Learning rate', 1, 6, step = 1)
        l_r = l_r * 1e-6

        n_c_images = st.number_input('Number of class images', step =1)

        s_b_size = st.number_input('Sample batch size', step =1)

        m_t_steps = st.number_input('Training steps', step =1)

        ft_start = st.button('Start fine-tuning')

        if ft_start and st.session_state.files_uploaded:   # Checks if any photos have been uploaded before starting fine-tuning, if not gives an error
          process = st.empty()            # Creating a Fine-tuning...
          process.text('Fine-tuning...')

          st.session_state.pipe_s = finetunesdcall(u_token, u_class, plant, t_b_size, l_r, n_c_images, s_b_size, m_t_steps, st.session_state.pictures)              # Calls dreambooth fine-tuning function
          st.session_state.fine_tuned = True

          process.empty()        # Deletes Fine-tuning... text
          st.write('Fine-tuning completed, now an image can be generated')
        elif ft_start and not st.session_state.files_uploaded:
          st.error('Please upload images to fine-tune the model', icon="🚨")

        guidance_scale = st.slider('Guidance scale', 1.0, 25.0, 7.5, step = 0.5, key = 12)

        num_inference_steps = st.number_input('Number of inference steps', step = 1, key = 13)

        use_seed = st.toggle('Enter a seed value', key = 14, help = 'If seed value not entered, each generated image will be given a random one.')
        if use_seed:
          s33d = st.number_input('Initial seed', value = 0, step = 1, key = 5)
        else:
          s33d = None

  with col2:
    st.title("Akbank Avatar Generator")
    use_db = st.toggle('Use Dreambooth')  # If you want to use Dreambooth, reveals the following untill the line:
    if use_db:
      st.write('Show us what you look like 😊')
      uploaded_files = st.file_uploader("Upload your photos here...", type=['png', 'jpg', 'jpeg'], accept_multiple_files=True, help = 'Please only upload JPEGs or PNGs of 512x512 pixels.')
      if uploaded_files:
        st.session_state.pictures = uploaded_files
        st.session_state.files_uploaded = True
      information = st.checkbox('See additional information for fine tuned image generation with Dreambooth')
      if information:               # Additional infromation toggle
        display_text = """
        Ideal parameters for cartoonish avatar generation:

        (Used 8 inputted training images)
        ▪️ Training batch size = 2 (effect not tested)
        ▪️ Learning rate = 1e-6
        ▪️ Number of class images = 100 (effect not tested)
        ▪️ Sample batch size = 4 (effect not tested)
        ▪️ Training steps = 640 and 664
          (The ideal number of training steps seems to be 80-83 epochs per image.)
        ▪️ Sample prompt = unique token + class of regularization images
        ▪️ Prompt = A cartoonish avatar version of Sample prompt.
        ▪️ Negative prompt = Up to the user
        ▪️ Guidance scale = 9
        ▪️ Number of inference steps = 100
        """
        st.text_area("Ideal parameters for different prompts and uses:", display_text)

      st.write(':red[Attention:] Dreambooth is currently only available for Stable-Diffusion-v1-5. When using Dreambooth this model is automatically implemented, no need to choose models 😅')
################################################################################
    option = st.selectbox('What model would you like to use?', ('Stable Diffusion v1-4', 'Stable Diffusion 2-1', 'FuseDream (CLIP+BigGAN)', 'Non-specified Flow based model'), placeholder = "Select your model...")

    user_prompt = st.text_input('Prompt', '', placeholder='* Required to fill this area.')

    user_n_prompt = st.text_input('Negative prompt (Optional)', '')
    st.write(':red[Attention:] Negative prompts are currently only available for diffusion models')

    generation_start = st.button('Start generating')
    if generation_start and user_prompt == '':          # Ensures a prompt is entered
      st.error('Please enter a prompt', icon="🚨")
    elif generation_start and user_prompt != '':

      select_sd = option == 'Stable Diffusion v1-4' or option == 'Stable Diffusion 2-1'

      process = st.empty()            # Creating a Generating...
      process.text('Generating...')

      if use_db:        # Selected model is Dreambooth fine tuning on Stable Diffusion v1-5
        if st.session_state.fine_tuned:
          generated_image = dbgeneratecall(st.session_state.pipe_s, user_prompt, user_n_prompt, guidance_scale, num_inference_steps, use_seed, s33d)

          process.empty()            # Deletes Generating... text
          st.write('')

          st.image(generated_image, caption= 'Generated image using Dreambooth fine-tuning.')       # Display the generated image with caption
        else:
          process.empty()            # Deletes Generating... text
          st.error('Please first fine-tune the model', icon="🚨")

      elif select_sd:       # Selected model is a Stable Diffusion one
        if not situation:
          i_inference_steps = 0         # If chosen to use default parameters, gives placeholder values that will be changed to default values within the stablediffusioncall function
          i_guidance_scale = 0
          i_initial_seed = 0

        generated_image = stablediffusioncall(option, user_prompt, user_n_prompt, i_inference_steps, i_initial_seed ,i_guidance_scale, situation)       # Calls stablediffusioncall function

        process.empty()            # Deletes Generating... text
        st.write('')

        st.image(generated_image, caption= f'Generated image using {option}.')       # Display the generated image with caption

      elif option == 'FuseDream (CLIP+BigGAN)':    # Selected model is FuseDream
        if not situation:
          INIT_ITERS =  1000
          OPT_ITERS = 1000                 # If chosen to use default parameters, gives default values
          NUM_BASIS = 10
          SEED = None
          use_seed = False

        generated_image = fusedreamcall(user_prompt, INIT_ITERS, OPT_ITERS, NUM_BASIS, SEED, use_seed)            # Calls fusedreamcall function

        process.empty()           # Deletes Generating... text
        st.write('')

        st.image(generated_image, caption= f'Generated image using {option}.')    # Display the generated image with caption

      elif option == 'Non-specified Flow based model':      # Selected model is a Non-specified Flow based model
        process.empty()                  # Deletes Generating... text
        st.write('')

        st.write('Error: This model is not implemented yet', key = 51)

  with col3:
    st.image(image2, caption='', width = 150, use_column_width=True, output_format="PNG")

###############################################################################################################

if __name__ == "__main__":
    main()


SyntaxError: invalid syntax (1228265611.py, line 3)

In [ ]:
!curl ipecho.net/plain   # -----> Enter with remote IP address

In [ ]:
!streamlit run akbankavatargenerator.py & npx localtunnel --port 8501

OLD VERSIONS:
-------------------------------

In [ ]:
# V1:

%%writefile akbankavatargenerator.py
#####################################
import streamlit as st
import base64
import torch
from PIL import Image
#####################################
# Functions:

def stablediffusioncall(option, user_prompt, user_n_prompt, i_inference_steps, i_initial_seed, i_guidance_scale, situation):
  from diffusers import StableDiffusionPipeline

  if option == "Stable Diffusion v1-4":        # Uses PNDM scheduler
    model_id = "CompVis/stable-diffusion-v1-4"
  elif option == "Stable Diffusion 2-1":
    model_id = "stabilityai/stable-diffusion-2-1" # Uses DDIM scheduler

  device = "cuda"
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
  pipe = pipe.to(device)

  prompt = user_prompt
  negative_prompt = user_n_prompt

  if situation:
    num_inference_steps = i_inference_steps
    guidance_scale = i_guidance_scale

    if i_initial_seed is not None:
      generator = torch.Generator("cuda").manual_seed(i_initial_seed)
      image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, negative_prompt=negative_prompt, generator=generator ).images[0]
    else:
      image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, negative_prompt=negative_prompt).images[0]
  else:
    image = pipe(prompt=prompt, negative_prompt=negative_prompt).images[0]

  del StableDiffusionPipeline
  return image

def fusedreamcall(user_prompt, INIT_ITERS, OPT_ITERS, NUM_BASIS, SEED, use_seed):
  from tqdm import tqdm
  from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
  import torchvision
  import BigGAN_utils.utils as utils
  import clip
  import torch.nn.functional as F
  from DiffAugment_pytorch import DiffAugment
  import numpy as np
  from fusedream_utils import FuseDreamBaseGenerator, get_G
  import sys

  sentence = user_prompt

  if use_seed:
    utils.seed_rng(SEED)

  sys.argv = ['']

  G, config = get_G(512)
  generator = FuseDreamBaseGenerator(G, config, 10)
  z_cllt, y_cllt = generator.generate_basis(sentence, init_iters=INIT_ITERS, num_basis=NUM_BASIS)

  z_cllt_save = torch.cat(z_cllt).cpu().numpy()
  y_cllt_save = torch.cat(y_cllt).cpu().numpy()
  img, z, y = generator.optimize_clip_score(z_cllt, y_cllt, sentence, latent_noise=False, augment=True, opt_iters=OPT_ITERS, optimize_y=True)
  ### Set latent_noise = True yields slightly higher AugCLIP score, but slightly lower image quality.

  image = img
  image = (image / 2 + 0.5).clamp(0, 1)
  image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
  images = (image * 255).round().astype("uint8")
  pil_images = [Image.fromarray(image) for image in images]
  image = pil_images[0]

  del tqdm
  del torchvision
  del utils
  del clip
  del F
  del DiffAugment
  del FuseDreamBaseGenerator
  del get_G
  return image

#####################################

def main():
  #icon = Image.open('/content/icon.png')                                       # Remove comment
  st.set_page_config(layout="wide")                                             # Add page_icon = icon within the (), before layout

  #background_image = Image.open('/content/background.jpg')                     # Remove comment


  ##################################### For red background (Does not work):
  page_bg_img = '''
  <style>
  body {
  background-image: background_image
  background-size: cover;
  }
  </style>
  '''
  st.markdown(page_bg_img, unsafe_allow_html=True)
  #####################################

  col1, coli1, col2, coli2, col3, = st.columns((1,0.25,2,0.25,1))

  #image = Image.open('/content/akbank.jpg')                                    # Remove comment
  #image2 = Image.open('/content/sabancı.png')                                  # Remove comment

  with col1:
    #st.image(image, caption='', width = 500, use_column_width=True)            # Remove comment
    with st.sidebar:
      st.title(':red[MODELS]')
      st.header('General model information', divider='red')

      with st.expander("Stable Diffusion v1-4"):
        st.write("PLaceholder.")

      with st.expander("Stable Diffusion 2-1"):
        st.write("PLaceholder.")

      with st.expander("FuseDream (CLIP+BigGAN)"):
        st.write("PLaceholder.")

      with st.expander("Non-specified Flow based model"):
        st.write("PLaceholder.")

      st.header('Settings to tweak the models', divider='red')

      situation = st.toggle('Use custom parameters for models')
      if situation:
        with st.expander('Diffusion model parameters:'):

          i_inference_steps = st.number_input('Number of inference steps', step = 1)

          use_seed = st.toggle('Enter a seed value', help = 'If seed value not entered, each generated image will be given a random one.')
          if use_seed:
            i_initial_seed = st.number_input('Initial seed', value = 0, step = 1)
          else:
            i_initial_seed = None

          i_guidance_scale = st.slider('Guidance scale', 1.0, 50.0, 7.5, step = 0.5)

        with st.expander('Dreambooth parameters:'):

          id_inference_steps = st.number_input('Number of inference steps', step =1, key = 1)

          use_seed = st.toggle('Enter a seed value', key = 2, help = 'If seed value not entered, each generated image will be given a random one.')
          if use_seed:
            id_initial_seed = st.number_input('Initial seed', value = 0, step = 1, key = 3)
          else:
            id_initial_seed = None

          id_learning_rate = st.slider('Learning rate', 1, 6, step = 1)
          id_learning_rate = id_learning_rate * 1e-6

          id_training_steps = st.number_input('Training steps', step =1)

          id_save_interval = st.number_input('Save interval', step =1, help = 'Should be lower than the number of training steps!')

          id_train_batch_size = st.number_input('Training batch size', step =1, help = 'Should be at least 1 and at most the number of images you have uploaded!')

          id_num_class_images = st.number_input('Number of classification images', step =1)

        with st.expander('FuseDream (CLIP+BigGAN):'):

          INIT_ITERS = st.number_input('Number of images used for initialization', step = 1)

          OPT_ITERS = st.number_input('Number of iterations', step = 1)

          NUM_BASIS = st.number_input('Number of basis images', step = 1)

          use_seed = st.toggle('Enter a seed value', key = 4, help = 'If seed value not entered, each generated image will be given a random one.')
          if use_seed:
            SEED = st.number_input('Initial seed', value = 0, step = 1, key = 5)
          else:
            SEED = None

        with st.expander('Non-specified Flow based model parameters:'):
          st.write("PLaceholder.")

  with col2:
    st.title("Akbank Avatar Generator")

    st.write('Show us what you look like 😊')
    uploaded_file = st.file_uploader("Upload your photos here...", type=['png', 'jpg', 'jpeg'], accept_multiple_files=True, help = 'Please only upload JPEGs or PNGs of 512x512 pixels.')
    st.write('Attention: Fine tuning with external data is currently only available for diffusion models 😅')

    option = st.selectbox('What model would you like to use?', ('Stable Diffusion v1-4', 'Stable Diffusion 2-1', 'FuseDream (CLIP+BigGAN)', 'Non-specified Flow based model'), placeholder = "Select your model...")

    user_prompt = st.text_input('Prompt', '', placeholder='* Required to fill this area.')

    user_n_prompt = st.text_input('Negative prompt (Optional)', '')
    st.write('Attention: Negative prompts are currently only available for diffusion models')

    generation_start = st.button('Start generating')
    if generation_start and user_prompt == '':
      st.error('Please enter a prompt', icon="🚨")
    elif generation_start and user_prompt != '':
      process = st.empty()
      process.text('Generating...')

      if option == 'Stable Diffusion v1-4' or option == 'Stable Diffusion 2-1':
        if not situation:
          i_inference_steps = 0
          i_guidance_scale = 0
          i_initial_seed = 0

        generated_image = stablediffusioncall(option, user_prompt, user_n_prompt, i_inference_steps, i_initial_seed ,i_guidance_scale, situation)

        process.empty()            # Until the other models are implemented keep these 3 rows here. After, place them just before with col3, lined up to the beggining of the above elif.
        st.write('')

        st.image(generated_image, caption= f'Generated image using {option}.')

      elif option == 'FuseDream (CLIP+BigGAN)':
        if not situation:
          INIT_ITERS =  1000
          OPT_ITERS = 1000
          NUM_BASIS = 10
          SEED = None
          use_seed = False

        generated_image = fusedreamcall(user_prompt, INIT_ITERS, OPT_ITERS, NUM_BASIS, SEED, use_seed)

        process.empty()            # Until the other models are implemented keep these 3 rows here. After, place them just before with col3, lined up to the beggining of the above elif.
        st.write('')

        st.image(generated_image, caption= f'Generated image using {option}.')

      elif option == 'Non-specified Flow based model':
        st.write('This model is not implemented yet', key = 51)

  with col3:
    for i in range(60):
      st.write('')
    #st.image(image2, caption='', width = 150, use_column_width=False, output_format="PNG")              # Remove comment

#####################################

if __name__ == "__main__":
    main()


In [ ]:
# V2: (Shelved)

%%writefile akbankavatargenerator.py
#####################################
import streamlit as st
import base64
import torch
from PIL import Image
#####################################
# Functions:

# Function for calling any of the two stable diffusion models, imports and deletes required modules, parameters are self explanatory, returns the image:
def stablediffusioncall(option, user_prompt, user_n_prompt, i_inference_steps, i_initial_seed, i_guidance_scale, situation):
  from diffusers import StableDiffusionPipeline

  if option == "Stable Diffusion v1-4":        # Uses a PNDM scheduler
    model_id = "CompVis/stable-diffusion-v1-4"
  elif option == "Stable Diffusion 2-1":
    model_id = "stabilityai/stable-diffusion-2-1" # Uses a DDIM scheduler

  device = "cuda"
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
  pipe = pipe.to(device)

  prompt = user_prompt
  negative_prompt = user_n_prompt

  if situation:     # If custom parameters are chosen, parameters are given the user input values. If not, the pipeline is called without change.
    num_inference_steps = i_inference_steps
    guidance_scale = i_guidance_scale

    if i_initial_seed is not None:        # In case of wanting to use custom seed, use the user input value. If not the pipeline is called without seed input.
      generator = torch.Generator("cuda").manual_seed(i_initial_seed)
      image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, negative_prompt=negative_prompt, generator=generator ).images[0]
    else:
      image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, negative_prompt=negative_prompt).images[0]
  else:
    image = pipe(prompt=prompt, negative_prompt=negative_prompt).images[0]

  del StableDiffusionPipeline
  return image

# Function for calling FuseDream model, imports and deletes required modules, parameters are self explanatory returns the image:
def fusedreamcall(user_prompt, INIT_ITERS, OPT_ITERS, NUM_BASIS, SEED, use_seed):
  from tqdm import tqdm
  from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
  import torchvision
  import BigGAN_utils.utils as utils
  import clip
  import torch.nn.functional as F
  from DiffAugment_pytorch import DiffAugment
  import numpy as np
  from fusedream_utils import FuseDreamBaseGenerator, get_G
  import sys

  sentence = user_prompt

  if use_seed:           # If the function is called using default parameter values SEED and use_seed are given the value None. If the utils.seed_rng(SEED) is called with None it gives an error, therefore the if statment is used.
    utils.seed_rng(SEED)

  sys.argv = ['']

  G, config = get_G(512)
  generator = FuseDreamBaseGenerator(G, config, 10)
  z_cllt, y_cllt = generator.generate_basis(sentence, init_iters=INIT_ITERS, num_basis=NUM_BASIS)

  z_cllt_save = torch.cat(z_cllt).cpu().numpy()
  y_cllt_save = torch.cat(y_cllt).cpu().numpy()
  img, z, y = generator.optimize_clip_score(z_cllt, y_cllt, sentence, latent_noise=False, augment=True, opt_iters=OPT_ITERS, optimize_y=True)

  image = img
  image = (image / 2 + 0.5).clamp(0, 1)
  image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
  images = (image * 255).round().astype("uint8")
  pil_images = [Image.fromarray(image) for image in images]
  image = pil_images[0]

  del tqdm
  del torchvision
  del utils
  del clip
  del F
  del DiffAugment
  del FuseDreamBaseGenerator
  del get_G
  return image

###############################################################################################################

def main():
  icon = Image.open('/content/icon.png')
  st.set_page_config(page_icon = icon, layout="wide")

  #background_image = Image.open('/content/background.jpg')

  col1, coli1, col2, coli2, col3, = st.columns((1,0.25,2,0.25,1))

  image = Image.open('/content/akbank.jpg')
  image2 = Image.open('/content/sabancı.png')

  with col1:
    st.image(image, caption='', width = 500, use_column_width=True)
    with st.sidebar:
      st.title(':red[MODELS]')
      st.header('General model information', divider='red')

      with st.expander("Stable Diffusion v1-4"):
        st.write("PLaceholder.")

      with st.expander("Stable Diffusion 2-1"):
        st.write("PLaceholder.")

      with st.expander("FuseDream (CLIP+BigGAN)"):
        st.write("PLaceholder.")

      with st.expander("Non-specified Flow based model"):
        st.write("PLaceholder.")

      st.header('Settings to tweak the models', divider='red')
      # situation = True --> Want to use custom parameters, situation = False --> Want to use default parameters
      situation = st.toggle('Use custom parameters for models')
      if situation:
        with st.expander('Diffusion model parameters:'):  # Have 3 parameters

          i_inference_steps = st.number_input('Number of inference steps', step = 1)

          use_seed = st.toggle('Enter a seed value', help = 'If seed value not entered, each generated image will be given a random one.')
          if use_seed:
            i_initial_seed = st.number_input('Initial seed', value = 0, step = 1)
          else:
            i_initial_seed = None

          i_guidance_scale = st.slider('Guidance scale', 1.0, 50.0, 7.5, step = 0.5)

        with st.expander('FuseDream (CLIP+BigGAN) parameters:'): # Has 5 parameters (including the boolean use_seed)

          INIT_ITERS = st.number_input('Number of images used for initialization', step = 1)

          OPT_ITERS = st.number_input('Number of iterations', step = 1)

          NUM_BASIS = st.number_input('Number of basis images', step = 1)

          use_seed = st.toggle('Enter a seed value', key = 4, help = 'If seed value not entered, each generated image will be given a random one.')
          if use_seed:
            SEED = st.number_input('Initial seed', value = 0, step = 1, key = 5)
          else:
            SEED = None

        with st.expander('Non-specified Flow based model parameters:'): # Not yet implemented, has 0 parameters
          st.write("PLaceholder.")

      st.header(' ', divider='red')

      with st.expander('Dreambooth parameters:'): # Has 9 parameters, Dreambooth is separated from the situation block since it should not be called with default parameters as the parameters need to be entered according to the external data and its use. (TLDR Attune parameters per each use for good results.)

        u_token = st.text_input('Unique token', '', placeholder='Enter a unique token.', help = 'example: name_surname or zwx')

        u_class = st.text_input('Class of reg images', '', placeholder='Enter a regularization image set class.', help = 'if restricted amount of available image sets, example: list of the available class names; else example: dog or person')

        id_inference_steps = st.number_input('Number of inference steps', step =1, key = 1)

        use_seed = st.toggle('Enter a seed value', key = 2, help = 'If seed value not entered, each generated image will be given a random one.')
        if use_seed:
          id_initial_seed = st.number_input('Initial seed', value = 0, step = 1, key = 3)
        else:
          id_initial_seed = None

        id_learning_rate = st.slider('Learning rate', 1, 6, step = 1)
        id_learning_rate = id_learning_rate * 1e-6

        id_training_steps = st.number_input('Training steps', step =1)

        id_save_interval = st.number_input('Save interval', step =1, help = 'Should be lower than the number of training steps!')

        id_train_batch_size = st.number_input('Training batch size', step =1, help = 'Should be at least 1 and at most the number of images you have uploaded!')

        id_num_class_images = st.number_input('Number of classification images', step =1)

  with col2:
    st.title("Akbank Avatar Generator")
    use_db = st.toggle('Use Dreambooth')  # If you want to use Dreambooth, reveals the following untill the line:
    if use_db:
      st.write('Show us what you look like 😊')
      uploaded_file = st.file_uploader("Upload your photos here...", type=['png', 'jpg', 'jpeg'], accept_multiple_files=True, help = 'Please only upload JPEGs or PNGs of 512x512 pixels.')
      information = st.checkbox('See additional information for fine tuned image generation')
      if information:               # Additional infromation toggle
        st.write('PLaceholder')
      st.write('Attention: Fine tuning with external data is currently only available for diffusion models 😅')
################################################################################
    option = st.selectbox('What model would you like to use?', ('Stable Diffusion v1-4', 'Stable Diffusion 2-1', 'FuseDream (CLIP+BigGAN)', 'Non-specified Flow based model'), placeholder = "Select your model...")

    user_prompt = st.text_input('Prompt', '', placeholder='* Required to fill this area.')

    user_n_prompt = st.text_input('Negative prompt (Optional)', '')
    st.write('Attention: Negative prompts are currently only available for diffusion models')

    generation_start = st.button('Start generating')
    if generation_start and user_prompt == '':          # Ensures a prompt is entered
      st.error('Please enter a prompt', icon="🚨")
    elif generation_start and user_prompt != '':

      select_sd = option == 'Stable Diffusion v1-4' or option == 'Stable Diffusion 2-1'

      if use_db and not select_sd:                                     # Ensures that a diffusion model is selected in the case of wanting to use Dreambooth
        st.error('Please select a diffusion model', icon="🚨")
      else:

        process = st.empty()            # Creating a Generating...
        process.text('Generating...')

        if use_db and select_sd:     # Selected model is a Stable Diffusion one with Dreambooth
          process.empty()            # Deletes Generating... text
          st.write('')

          st.write('Error: This model is not implemented yet', key = 51)

        elif select_sd:       # Selected model is a Stable Diffusion one
          if not situation:
            i_inference_steps = 0         # If chosen to use default parameters, gives placeholder values that will be changed to default values within the stablediffusioncall function
            i_guidance_scale = 0
            i_initial_seed = 0

          generated_image = stablediffusioncall(option, user_prompt, user_n_prompt, i_inference_steps, i_initial_seed ,i_guidance_scale, situation)       # Calls stablediffusioncall function

          process.empty()            # Deletes Generating... text
          st.write('')

          st.image(generated_image, caption= f'Generated image using {option}.')       # Display the generated image with caption

        elif option == 'FuseDream (CLIP+BigGAN)':    # Selected model is FuseDream
          if not situation:
            INIT_ITERS =  1000
            OPT_ITERS = 1000                 # If chosen to use default parameters, gives default values
            NUM_BASIS = 10
            SEED = None
            use_seed = False

          generated_image = fusedreamcall(user_prompt, INIT_ITERS, OPT_ITERS, NUM_BASIS, SEED, use_seed)            # Calls fusedreamcall function

          process.empty()           # Deletes Generating... text
          st.write('')

          st.image(generated_image, caption= f'Generated image using {option}.')    # Display the generated image with caption

        elif option == 'Non-specified Flow based model':      # Selected model is a Non-specified Flow based model
          process.empty()                  # Deletes Generating... text
          st.write('')

          st.write('Error: This model is not implemented yet', key = 51)

  with col3:
    st.image(image2, caption='', width = 150, use_column_width=True, output_format="PNG")

###############################################################################################################

if __name__ == "__main__":
    main()


SANDBOX:
---------------------------------------------------------------------

In [ ]:
# Dreambooth requirements:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton                                                 # use either this or the one at the bottom
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers

In [ ]:
!accelerate config default

In [ ]:
# Settings (including model selection):

u_token="yigitya"

u_class="person"

plant=1337

t_b_size=2

l_r=1e-6

n_c_images=100

s_b_size=4

m_t_steps=640

s_prompt=u_token+" "+u_class

MODEL_NAME = "runwayml/stable-diffusion-v1-5"

OUTPUT_DIR = "/content/stable_diffusion_weights/" + u_token

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR


In [ ]:
# Choosing the instance and class prompts:

# You can also add multiple concepts here, try tweaking `--max_train_steps` accordingly.
# `class_data_dir` contains the regularization images
concepts_list = [
    {
        "instance_prompt":      f"photo of {u_token} man",
        "class_prompt":         "photo of a man",
        "instance_data_dir":    "/content/data/" + u_token,
        "class_data_dir":       "/content/data/man"
    },
    {
        "instance_prompt":      f"photo of {u_token} woman",
        "class_prompt":         "photo of a woman",
        "instance_data_dir":    "/content/data/" + u_token,
        "class_data_dir":       "/content/data/woman"
    },
    {
        "instance_prompt":      f"photo of {u_token} person",
        "class_prompt":         "photo of a person",
        "instance_data_dir":    "/content/data/" + u_token,
        "class_data_dir":       "/content/data/person"
    }
]

import json
import os
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

keep_cl = []
for c in concepts_list:
  a_class = c['instance_prompt'].split()
  if a_class[-1] == u_class:
    keep_cl.append(c)

with open("concepts_list.json", "w") as f:
    json.dump(keep_cl, f, indent=4)

In [ ]:
# Upload your images by running this cell:

import os
from google.colab import files
import shutil

for c in concepts_list:
  a_class = c['instance_prompt'].split()
  if a_class[-1] == u_class:
    print(f"Uploading instance images for `{c['instance_prompt']}`")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(c['instance_data_dir'], filename)
        shutil.move(filename, dst_path)

In [ ]:
# Fine-tuning the SD model:

# Tweak the parameters for desired image quality --> 1200 training steps = 40 newly introdued images for 30 epochs.
!python3 train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=$plant \
  --resolution=512 \
  --train_batch_size=$t_b_size \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=$l_r \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=$n_c_images \
  --sample_batch_size=$s_b_size \
  --max_train_steps=$m_t_steps \
  --save_interval=10000 \
  --save_sample_prompt="$s_prompt" \
  --concepts_list="concepts_list.json"

# Reduce the `--save_interval` to lower than `--max_train_steps` to save weights from intermediate steps.
# `--save_sample_prompt` can be same as `--instance_prompt` to generate intermediate samples (saved along with weights in samples)

In [ ]:
# Specify the weights directory to use (leave blank for latest):

from natsort import natsorted
from glob import glob
import os
WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

In [ ]:
# Inference:

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = WEIGHTS_DIR

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = None

In [ ]:
# Can set a random seed here for reproducibility:

g_cuda = torch.Generator(device='cuda')
seed = 52362
g_cuda.manual_seed(seed)

In [ ]:
# Run for generating images.

prompt = "A version of yigitya person as an cosmonaut with the helmet and suit in outher space."
negative_prompt = "weird eyes, blurry eyes, fat "
num_samples = 1
guidance_scale = 9
num_inference_steps = 100
height = 512
width = 512

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)